### Data ingestion to vector db
#### Data parsing, chuncking, embedding, and vector storage

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
### Read all the pdfs inside directory

def process_all_pdfs(pdf_directory):
    """Process all the pdf files in a directory"""
    all_documents = []
    pdf_directory = Path(pdf_directory)

    pdf_files = list(pdf_directory.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing {pdf_file.name}...")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            ## add source info to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)

            print(f"Loaded {len(documents)} documents from {pdf_file}")

        except Exception as e:
            print(f"error: {e}")
    
    print(f"\ntotal documents loaded: {len(all_documents)}")
    return all_documents

all_pdf_documents = process_all_pdfs("../data/pdf")

Found 4 PDF files to process

Processing xgboost.pdf...
Loaded 13 documents from ../data/pdf/xgboost.pdf

Processing embeddings.pdf...
Loaded 16 documents from ../data/pdf/embeddings.pdf

Processing attention.pdf...
Loaded 11 documents from ../data/pdf/attention.pdf

Processing object_detection.pdf...
Loaded 21 documents from ../data/pdf/object_detection.pdf

total documents loaded: 61


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.12', 'creator': 'LaTeX with hyperref package', 'creationdate': '2016-06-14T01:29:40+00:00', 'author': '', 'keywords': '', 'moddate': '2016-06-14T01:29:40+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-2.3-1.40.12 (TeX Live 2011) kpathsea version 6.0.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/xgboost.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': 'xgboost.pdf', 'file_type': 'pdf'}, page_content='XGBoost: A Scalable Tree Boosting System\nTianqi Chen\nUniversity of Washington\ntqchen@cs.washington.edu\nCarlos Guestrin\nUniversity of Washington\nguestrin@cs.washington.edu\nABSTRACT\nTree boosting is a highly eﬀective and widely used machine\nlearning method. In this paper, we describe a scalable end-\nto-end tree boosting system called XGBoost, which is used\nwidely by data scientists to achieve state-of-the-art results\non many machine learning challenges. We propose a novel

In [4]:
### split texts and get into chuncks

def split_documents(documents, chunk_size=1000, overlap_size=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = overlap_size,
        length_function = len,
        separators= ["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(all_pdf_documents)
    print(f"Split {len(all_pdf_documents)} documents into {len(split_docs)} chunks")

    ## Example of chunck
    if split_docs:
        print("\nExample chunck: ")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs


In [5]:
chunks = split_documents(all_pdf_documents)

Split 61 documents into 401 chunks

Example chunck: 
Content: XGBoost: A Scalable Tree Boosting System
Tianqi Chen
University of Washington
tqchen@cs.washington.edu
Carlos Guestrin
University of Washington
guestrin@cs.washington.edu
ABSTRACT
Tree boosting is a h...
Metadata: {'producer': 'pdfTeX-1.40.12', 'creator': 'LaTeX with hyperref package', 'creationdate': '2016-06-14T01:29:40+00:00', 'author': '', 'keywords': '', 'moddate': '2016-06-14T01:29:40+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-2.3-1.40.12 (TeX Live 2011) kpathsea version 6.0.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/xgboost.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': 'xgboost.pdf', 'file_type': 'pdf'}


### Embedding and vectorstore

In [6]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
            Initialize the embedding manager with a pre-trained model
            Model Used: Huggingface model name for sentence transformer
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """ Load the sentence transformer model """
        try:
            print(f"Loading model: {self.model_name}...")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully whose embedding dimension is {self.model.get_sentence_embedding_dimension()}")

        except Exception as e:
            print(f"Error loading model {self.model_name} : {e}")
            raise
    
    def generate_embedding(self, texts:List[str]) -> np.ndarray:
        """ Generate embeddings for a list of texts using the sentence transformer model """

        if not self.model:
            raise ValueError("Model not loaded. Please call _load_model() first.")
        
        print(f"Generating embeddings for {len(texts)} texts...")

        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Embeddings generated successfully with shape {embeddings.shape}")

        return embeddings            

In [8]:
### initialise Embedding

embedding_manager = EmbeddingManager()

Loading model: sentence-transformers/all-MiniLM-L6-v2...


No sentence-transformers model found with name sentence-transformers/all-MiniLM-L6-v2. Creating a new one with mean pooling.


Model loaded successfully whose embedding dimension is 384


### VectorStore

In [9]:
class VectorStore:
    """ Manages document embeddings in vector database"""
    def __init__(self, collection_name:str = "pdf-documents", persistent_directory = "../data/vector_store"):
        """ Initialize the vector store with a collection name and persistent directory """
        self.collection_name = collection_name
        self.persistent_directory = persistent_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """ Initialize the chroma client """
        try:
            os.makedirs(self.persistent_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persistent_directory)
            self.collection = self.client.get_or_create_collection(name = self.collection_name,
                metadata = {"Description": "This is a collection of documents for RAG",
                "hnsw:space": "cosine"
                }
            )
            print(f"Vector store initialized successfully with collection {self.collection_name}")
            print(f"count of existing document in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents:List[Any], embeddings:np.ndarray):
        """ Add documents to the vector store 
        Arguments: 
            documents: List of langchain documents
            embeddings: numpy array of their embeddings
        """
        if len(documents)!=len(embeddings):
            raise ValueError("Number of documents should match with number of embeddings")
        
        print(f"Adding {len(documents)} documents to the vector store...")

        ## Prepare data to ingest in chroma db

        ids = []
        metadatas = []
        documents_text = []
        embedding_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            ## Generate unique id
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            ## Prepare metadata

            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['contect_length'] = len(doc.page_content)
            metadatas.append(metadata)

            ## Document content
            documents_text.append(doc.page_content)

            ## Embedding

            embedding_list.append(embedding.tolist())

        ## Add to collection

        try:
            self.collection.add(
                ids = ids,
                documents = documents_text,
                metadatas = metadatas,
                embeddings = embedding_list
            )
            print(f"{len(documents)} documents added successfully to the vector store")
            print(f"total documents in the collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to the vector store: {e}")
            raise
        

In [10]:
vectorstore = VectorStore()

Vector store initialized successfully with collection pdf-documents
count of existing document in collection: 0


In [11]:
vectorstore

In [12]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.12', 'creator': 'LaTeX with hyperref package', 'creationdate': '2016-06-14T01:29:40+00:00', 'author': '', 'keywords': '', 'moddate': '2016-06-14T01:29:40+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.1415926-2.3-1.40.12 (TeX Live 2011) kpathsea version 6.0.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/xgboost.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1', 'source_file': 'xgboost.pdf', 'file_type': 'pdf'}, page_content='XGBoost: A Scalable Tree Boosting System\nTianqi Chen\nUniversity of Washington\ntqchen@cs.washington.edu\nCarlos Guestrin\nUniversity of Washington\nguestrin@cs.washington.edu\nABSTRACT\nTree boosting is a highly eﬀective and widely used machine\nlearning method. In this paper, we describe a scalable end-\nto-end tree boosting system called XGBoost, which is used\nwidely by data scientists to achieve state-of-the-art results\non many machine learning challenges. We propose a novel

In [13]:
### Convert text to embeddings
texts = [doc.page_content for doc in chunks]

In [14]:
## Generate Embeddings
embeddings = embedding_manager.generate_embedding(texts)

## Add embeddings to vector store

vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 401 texts...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]

Embeddings generated successfully with shape (401, 384)
Adding 401 documents to the vector store...
401 documents added successfully to the vector store
total documents in the collection: 401


In [40]:
class RAGRetriever:
    """Handles query based retrival from vector store"""
    def __init__(self, vectorstore:VectorStore, embedding_manager: EmbeddingManager):

        self.vectorstore = vectorstore
        self.embedding_manager = embedding_manager

    def retrieve(self, query:str, top_k:int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        
        print(f"Retrieving top {top_k} documents for query: {query}")
        print(f"Top K: {top_k}, Score Threshold: {score_threshold}")

        query_embedding = self.embedding_manager.generate_embedding([query])[0]

        ## Perform similarity search
        try:
            results = self.vectorstore.collection.query(
                query_embeddings = [query_embedding.tolist()],
                n_results = top_k
            )
            # print(results)

            ## Process results

            retrived_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                scores = results['distances'][0]
                ids = results['ids'][0]

                # print(f"Documents: {documents}")
                # print(f"Metadatas: {metadatas}")
                # print(f"Scores: {scores}")
                # print(f"IDs: {ids}")

                for i, (doc, metadata, score, id) in enumerate(zip(documents, metadatas, scores, ids)):
                    similarity_score = 1 - (score / 2)

                    if similarity_score >= score_threshold:
                        retrived_docs.append({
                            "id": id,
                            "content": doc,
                            "metadata": metadata,
                            "similarity_score": similarity_score,
                            "distance": score,
                            "rank": i+1
                        })
                
                print(f"Retrived {len(retrived_docs)} documents with score greater than {score_threshold}")
            else:
                print("No documents found in the vector store")
            
            return retrived_docs
                    
        except Exception as e:
            print(f"Error during Retrival: {e}")
            return []


In [41]:
rag_retriver = RAGRetriever(vectorstore, embedding_manager)

In [42]:
rag_retriver

In [44]:
rag_retriver.retrieve("what is role of Deep learning in Pedestrian Detection")

Retrieving top 5 documents for query: what is role of Deep learning in Pedestrian Detection
Top K: 5, Score Threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generated successfully with shape (1, 384)
Retrived 5 documents with score greater than 0.0


[{'id': 'doc_399f4332_200',
  'content': 'ditions. It should be noticed that the covered domains are\ndiversiﬁed. Pedestrian and face images have regular structures,\nwhile general objects and scene images have more complex\nvariations in geometric structures and layouts. Therefore,\ndifferent deep models are required by various images.\nThere has been a relevant pioneer effort [34] which mainly\nfocuses on relevant software tools to implement deep learning\ntechniques for image classiﬁcation and object detection, but\npays little attention on detailing speciﬁc algorithms. Different\nfrom it, our work not only reviews deep learning based object\ndetection models and algorithms covering different applica-\ntion domains in detail, but also provides their corresponding\nexperimental comparisons and meaningful analyses.\nThe rest of this paper is organized as follows. In Section\n2, a brief introduction on the history of deep learning and the\nbasic architecture of CNN is provided. Generic

#### Integration of context to LLM

In [76]:
### Simple RAG pipeline using GROQ API

from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(api_key=groq_api_key, model_name="llama-3.1-8b-instant", temperature=0, max_tokens = 1024)

def rag_simple(query:str, retriever, llm, top_k:int = 5):
    """ Simple RAG pipeline """
    print(f"Query: {query}")
    print(f"Top K: {top_k}")

    ## Retrieve
    results = retriever.retrieve(query, top_k=top_k)

    context = "\n\n".join([result['content'] for result in results]) if results else ""

    if not context:
        return "No relevant context found in the document"

    ## Generate response using Groq LLM
    prompt = f""" Use the following question to answer the question concisely.

        context: {context}

        question: {query}

        Answer:
        """

    response = llm.invoke(prompt)
    

    return response.content


In [77]:
answer = rag_simple("why is xgboost so popular", rag_retriver, llm)
print(answer)

Query: why is xgboost so popular
Top K: 5
Retrieving top 5 documents for query: why is xgboost so popular
Top K: 5, Score Threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Embeddings generated successfully with shape (1, 384)
Retrived 5 documents with score greater than 0.0
XGBoost is so popular due to its scalability in all scenarios, running more than ten times faster than existing popular solutions on a single machine and scaling to billions of examples in distributed or memory-limited settings. Its success can be attributed to several key innovations, including:

1. A novel tree learning algorithm for handling sparse data.
2. A theoretically justified weighted quantile sketch procedure for handling instance weights.
3. Parallel and distributed computing, making learning faster and enabling quicker model exploration.
4. The ability to exploit out-of-core data sets with given resources.

These innovations, combined with its ability to handle large datasets and provide state-of-the-art results on a wide range of problems, have made XGBoost the consensus choice of learner in many competitions and real-world applications.
